In [1]:
# !pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp
import cv2
import numpy as np  

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
VIDEO_PATH = 'videoData/deadlift/lean.avi'
EXPORT_PATH = 'annotations/deadlift/lean.csv'
MODEL_PATH = 'models/deadlift/lean.pkl'

1.1 Save Video

In [41]:
cap = cv2.VideoCapture(0)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter(VIDEO_PATH, cv2.VideoWriter_fourcc('P', 'I', 'M', '1'), fps, (int(width), int(height)))

cv2.VideoCapture()

while cap.isOpened():
    ret,frame = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    try:
        cv2.imshow('Press', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


2. Capture Landmars and Export to CSV

In [42]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [43]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [44]:
with open(EXPORT_PATH, mode='w', newline='') as f:
    csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [45]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist() #Converted to a list
        # print(keypoints)
        keypoints.insert(0,action)
        
        with open(EXPORT_PATH, mode='a', newline='') as f:
            csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

In [46]:
cap = cv2.VideoCapture(VIDEO_PATH) #name of the video
#Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        #Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detection
        results = pose.process(image)

        #Recolor back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                )

        k = cv2.waitKey(1)
        if k== ord('a'):
            export_landmark(results,'right')
        elif k== ord('s'):
            export_landmark(results,'neutral')
        elif k== ord('d'):
            export_landmark(results,'left')
        
        cv2.imshow('Lean Webcam Feed',image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Can't receive frame (stream end?). Exiting ...


3. Train Custom Model Using Scikit Learn

3.1 Read in Collected Data and Process

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv(EXPORT_PATH)

In [6]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,neutral,0.509132,0.096270,-0.554616,0.999989,0.523320,0.071684,-0.520357,0.999979,0.530116,...,0.380758,0.085903,0.584484,1.234880,0.315110,0.060184,0.457711,1.268279,0.157262,0.083754
1,neutral,0.509662,0.094453,-0.612368,0.999991,0.523437,0.070411,-0.579609,0.999982,0.530196,...,0.364934,0.093678,0.582518,1.229790,0.316927,0.062813,0.460747,1.265550,0.145268,0.087252
2,neutral,0.509697,0.094437,-0.621545,0.999990,0.523531,0.070392,-0.588355,0.999981,0.530333,...,0.388340,0.092600,0.582594,1.230423,0.329248,0.062846,0.461165,1.266128,0.171868,0.086978
3,neutral,0.510066,0.094425,-0.611409,0.999990,0.523842,0.070352,-0.578077,0.999980,0.530759,...,0.394742,0.090844,0.584845,1.243405,0.328181,0.062548,0.462093,1.274987,0.173927,0.086027
4,neutral,0.510144,0.094546,-0.605326,0.999989,0.523892,0.070327,-0.571568,0.999979,0.530813,...,0.391695,0.088382,0.585044,1.251028,0.333843,0.061594,0.464316,1.283763,0.170099,0.084751


In [7]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
1160,neutral,0.501561,0.097660,-0.661922,0.999968,0.516751,0.074708,-0.627230,0.999941,0.524458,...,0.419594,0.079247,0.548407,1.266399,0.280952,0.057471,0.441636,1.287277,0.179038,0.072230
1161,neutral,0.499585,0.097125,-0.660185,0.999969,0.514618,0.073482,-0.625209,0.999943,0.522425,...,0.423360,0.078346,0.547801,1.279169,0.331420,0.057507,0.437707,1.300406,0.179526,0.072443
1162,neutral,0.498286,0.096400,-0.668768,0.999970,0.512952,0.072385,-0.634988,0.999944,0.520592,...,0.424814,0.077775,0.545506,1.295540,0.332447,0.056763,0.433652,1.312694,0.178909,0.071938
1163,neutral,0.492700,0.091590,-0.671689,0.999968,0.506975,0.068110,-0.638085,0.999942,0.514825,...,0.452862,0.067929,0.521544,1.338130,0.349410,0.052034,0.421285,1.348087,0.182937,0.062780
1164,neutral,0.489478,0.091238,-0.675642,0.999968,0.503949,0.067865,-0.643418,0.999942,0.512027,...,0.443840,0.068937,0.523438,1.330182,0.331930,0.051502,0.418342,1.342844,0.194048,0.062772


In [8]:
df[df['class'] == 'up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33


In [9]:
X = df.drop('class', axis=1) #Features
Y = df['class'] #Target value

In [53]:
X_train, X_test, y_train,y_test = train_test_split(X,Y,test_size = 0.3, random_state = 1234)

In [54]:
X

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,0.509132,0.096270,-0.554616,0.999989,0.523320,0.071684,-0.520357,0.999979,0.530116,0.071978,...,0.380758,0.085903,0.584484,1.234880,0.315110,0.060184,0.457711,1.268279,0.157262,0.083754
1,0.509662,0.094453,-0.612368,0.999991,0.523437,0.070411,-0.579609,0.999982,0.530196,0.070766,...,0.364934,0.093678,0.582518,1.229790,0.316927,0.062813,0.460747,1.265550,0.145268,0.087252
2,0.509697,0.094437,-0.621545,0.999990,0.523531,0.070392,-0.588355,0.999981,0.530333,0.070751,...,0.388340,0.092600,0.582594,1.230423,0.329248,0.062846,0.461165,1.266128,0.171868,0.086978
3,0.510066,0.094425,-0.611409,0.999990,0.523842,0.070352,-0.578077,0.999980,0.530759,0.070720,...,0.394742,0.090844,0.584845,1.243405,0.328181,0.062548,0.462093,1.274987,0.173927,0.086027
4,0.510144,0.094546,-0.605326,0.999989,0.523892,0.070327,-0.571568,0.999979,0.530813,0.070695,...,0.391695,0.088382,0.585044,1.251028,0.333843,0.061594,0.464316,1.283763,0.170099,0.084751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,0.501561,0.097660,-0.661922,0.999968,0.516751,0.074708,-0.627230,0.999941,0.524458,0.075759,...,0.419594,0.079247,0.548407,1.266399,0.280952,0.057471,0.441636,1.287277,0.179038,0.072230
1161,0.499585,0.097125,-0.660185,0.999969,0.514618,0.073482,-0.625209,0.999943,0.522425,0.074575,...,0.423360,0.078346,0.547801,1.279169,0.331420,0.057507,0.437707,1.300406,0.179526,0.072443
1162,0.498286,0.096400,-0.668768,0.999970,0.512952,0.072385,-0.634988,0.999944,0.520592,0.073544,...,0.424814,0.077775,0.545506,1.295540,0.332447,0.056763,0.433652,1.312694,0.178909,0.071938
1163,0.492700,0.091590,-0.671689,0.999968,0.506975,0.068110,-0.638085,0.999942,0.514825,0.069190,...,0.452862,0.067929,0.521544,1.338130,0.349410,0.052034,0.421285,1.348087,0.182937,0.062780


3.2 Train Machine Learning Classification Model

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [11]:
pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier()),
}

In [12]:
fit_models = {}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

NameError: name 'X_train' is not defined

In [58]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

3.3 Evaluate and Serialize Model


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score #Accuracy metrics
import pickle

In [60]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test.values,yhat),
            precision_score(y_test.values,yhat,average='weighted'),
            recall_score(y_test.values,yhat,average='weighted'))

lr 0.9971428571428571 0.9971603856266433 0.9971428571428571
rc 1.0 1.0 1.0
rf 0.9971428571428571 0.9971603856266433 0.9971428571428571
gb 0.9942857142857143 0.994355400696864 0.9942857142857143


In [61]:
yhat = fit_models['rc'].predict(X_test)

In [62]:
yhat[:10]

array(['left', 'neutral', 'right', 'left', 'neutral', 'neutral', 'left',
       'right', 'right', 'right'], dtype='<U7')

In [63]:
y_test

378        left
95      neutral
1004      right
828        left
324     neutral
         ...   
647        left
959       right
954       right
847        left
871     neutral
Name: class, Length: 350, dtype: object

In [64]:
with open(MODEL_PATH,'wb') as f:  #dumping the model
    pickle.dump(fit_models['rc'],f)

4. Make Detections with Model

In [14]:
with open(MODEL_PATH,'rb') as f:
    model = pickle.load(f)

In [ ]:
# X = pd.DataFrame([row], columns=landmarks[1:])

In [15]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = '' 
#Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        #Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detections
        results = pose.process(image)

        #Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Extract landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                )

        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            body_language_class = model.predict(X)[0]

            # For Random Forest
            # body_language_prob = model.predict_proba(X)[0]

            # For ridge classifier
            probs = model.decision_function(X)[0]
            body_language_prob = np.exp(probs) / np.sum(np.exp(probs))

            print(body_language_class, body_language_prob)

            if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = 'down'
            elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = 'up'
                counter += 1
                print(current_stage,counter)

            #Get status box
            cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)

            #Display Class
            cv2.putText(image,'CLASS',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,body_language_class.split(' ')[0],(90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Probability
            cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            # cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2)),(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Counter
            cv2.putText(image,'COUNTER',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        except Exception as e:
            print(e)
            pass
        
        cv2.imshow('Raw Webcam Feed',image)

        if(cv2.waitKey(10) & 0xFF == ord('q')):
            break

cap.release()
cv2.destroyAllWindows()


name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'landmarks' is not defined
name 'la